# # 문제 03 유형(DataSet_03.csv 이용)
 
 구분자 : comma(“,”), 5,001 Rows, 8 Columns, UTF-8 인코딩
 안경 체인을 운영하고 있는 한 회사에서 고객 사진을 바탕으로 안경의 사이즈를 맞춤 제작하는 비즈니스를 기획하고 있다. 우선 데이터만으로 고객의 성별을 파악하는 것이 가능할 지를 연구하고자 한다.

 컬 럼 / 정 의 / Type
 long_hair / 머리카락 길이 (0 – 길지 않은 경우 / 1 – 긴
 경우) / Integer
 forehead_width_cm / 이마의 폭 (cm) / Double
 forehead_height_cm / 이마의 높이 (cm) / Double
 nose_wide / 코의 넓이 (0 – 넓지 않은 경우 / 1 – 넓은 경우) / Integer
 nose_long / 코의 길이 (0 – 길지 않은 경우 / 1 – 긴 경우) / Integer
 lips_thin / 입술이 얇은지 여부 0 – 얇지 않은 경우 / 1 –
 얇은 경우) / Integer
 distance_nose_to_lip_long / 인중의 길이(0 – 인중이 짧은 경우 / 1 – 인중이
 긴 경우) / Integer
 gender / 성별 (Female / Male) / String

In [1]:
import pandas as pd
import numpy as np

data03 = pd.read_csv('C:/Users/joey0/CodeStudy/proDS/Dataset/Dataset_03.csv')

## 1.이마의 폭(forehead_width_cm)과 높이(forehead_height_cm) 사이의 비율(forehead_ratio)에 대해서 평균으로부터 3 표준편차 밖의 경우를 이상치로 정의할 때, 이상치에 해당하는 데이터는 몇 개인가?

In [2]:
q1 = data03.copy()

q1['forehead_ratio'] = q1['forehead_width_cm'] / q1['forehead_height_cm']

xbar = q1['forehead_ratio'].mean()
std = q1['forehead_ratio'].std()

UB = xbar + (3 * std)
LB = xbar - (3 * std)

len(q1[(q1['forehead_ratio'] > UB) | (q1['forehead_ratio'] < LB)])

#정답 : 3

3

## 2.성별에 따라 forehead_ratio 평균에 차이가 있는지 적절한 통계 검정을 수행하시오.
 - 검정은 이분산을 가정하고 수행한다.
 - 검정통계량의 추정치는 절대값을 취한 후 소수점 셋째 자리까지 반올림하여 기술하시오.
 - 신뢰수준 99%에서 양측 검정을 수행하고 결과는 귀무가설 기각의 경우 Y로, 그렇지 않을 경우 N으로 답하시오.

In [3]:
from scipy.stats import ttest_ind, ttest_rel

q1.columns

Index(['long_hair', 'forehead_width_cm', 'forehead_height_cm', 'nose_wide',
       'nose_long', 'lips_thin', 'distance_nose_to_lip_long', 'gender',
       'forehead_ratio'],
      dtype='object')

In [9]:
q1.gender.unique()

g_m = q1[q1.gender == 'Male']['forehead_ratio']
g_f = q1[q1.gender == 'Female']['forehead_ratio']

q2_out = ttest_ind(g_m, g_f, equal_var=False)
#Ttest_indResult(statistic=2.9994984197511543, pvalue=0.0027186702390657176)

round(q2_out.statistic, 3) #2.999
q2_out.pvalue < 0.01 #True

#정답 : 2.999, Y

True

## 3.주어진 데이터를 사용하여 성별을 구분할 수 있는지 로지스틱 회귀분석을 적용하여 알아 보고자 한다. 
 - 데이터를 7대 3으로 나누어 각각 Train과 Test set로 사용한다. 이 때 seed는 123으로 한다.
 - 원 데이터에 있는 7개의 변수만 Feature로 사용하고 gender를 label로 사용한다. (forehead_ratio는 사용하지 않음)
 - 로지스틱 회귀분석 예측 함수와 Test dataset를 사용하여 예측을 수행하고 정확도를 평가한다. 이 때 임계값은 0.5를 사용한다. 
 - Male의 Precision 값을 소수점 둘째 자리까지 반올림하여 기술하시오.

In [10]:
q3 = data03.copy()

q3.columns
#['long_hair', 'forehead_width_cm', 'forehead_height_cm', 'nose_wide', 'nose_long', 'lips_thin', 'distance_nose_to_lip_long', 'gender']

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, classification_report

In [13]:
train, test = train_test_split(q3, test_size=0.3, random_state=123)

var_list = q3.columns.drop('gender')

logit = LogisticRegression().fit(train[var_list], train['gender'])

q3_pred = logit.predict(test[var_list])

dir(logit)
logit.coef_

q3_pred_pr = logit.predict_proba(test[var_list])

precision_score(test['gender'], q3_pred, pos_label='Male')

print(classification_report(test['gender'], q3_pred))

#정답 : 0.96

              precision    recall  f1-score   support

      Female       0.97      0.96      0.96       743
        Male       0.96      0.97      0.97       758

    accuracy                           0.97      1501
   macro avg       0.97      0.97      0.97      1501
weighted avg       0.97      0.97      0.97      1501

